In [2]:
from keras.models import load_model
import tensorflow as tf
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware

from pydantic import BaseModel

from json import loads

from keras.models import load_model

import tensorflow as tf
import numpy as np
import spacy

from Objects.Functions.get_word_pairs import get_wp_in_line_hard
import pandas as pd

Neural Web

In [2]:
nlp_type_of_event_extraction = spacy.load('../../PipeLines/WordsExtraction/words_extract/')
nlp_classic = spacy.load('ru_core_news_sm')
kcm_extraction_model = tf.keras.models.load_model(
    '../../PipeLines/Coder/coder_from_spacy_to_kcm_onh5.h5')
# kcm_extraction_model.evaluate
is_description_model = load_model('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')

Pairs

In [3]:
def get_vectorized_wp_and_wp(text, vectorizer):
    list_of_wp = list()
    list_of_vectors = list()
    for sentence in text.split('.'):
        for wp in get_wp_in_line_hard(sentence):
            list_of_vectors.append(vectorizer(wp))
            list_of_wp.append(wp)
    return np.array(list_of_wp), np.array(list_of_vectors)

In [4]:
get_wp_in_line_hard("Мы-некоммерческий проект \"Наставник будущего\" при поддержке школ Москвы.")

['Мы-некоммерческий проект',
 'Мы-некоммерческий проект "Наставник',
 'Мы-некоммерческий проект "Наставник будущего"',
 'Мы-некоммерческий проект "Наставник будущего" при',
 'Мы-некоммерческий проект "Наставник будущего" при поддержке',
 'Мы-некоммерческий проект "Наставник будущего" при поддержке школ',
 'проект "Наставник',
 'проект "Наставник будущего"',
 'проект "Наставник будущего" при',
 'проект "Наставник будущего" при поддержке',
 'проект "Наставник будущего" при поддержке школ',
 '"Наставник будущего"',
 '"Наставник будущего" при',
 '"Наставник будущего" при поддержке',
 '"Наставник будущего" при поддержке школ',
 'будущего" при',
 'будущего" при поддержке',
 'будущего" при поддержке школ',
 'при поддержке',
 'при поддержке школ',
 'поддержке школ']

In [5]:
def sendStudentJson(description=Body()):
    results = dict()
    results['id'] = description['id']
    types_of_events = list(map(str, nlp_type_of_event_extraction(description['name_of_event']).ents))
    results['types_of_events'] = types_of_events
    wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(description['event_description'], lambda text: nlp_classic(text).vector)
    is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized) > 0.992
    # print(is_wp_descriptor)
    descriptors_vectorized = wp_of_event_vectorized[is_wp_descriptor.T[0]]
    descriptors_name = wp_of_event[is_wp_descriptor.T[0]]
    # print(descriptors_name)
    predict = kcm_extraction_model.predict(descriptors_vectorized) 
    skills = dict()
    for name, kcm in zip(descriptors_name, predict):
        skills[name] = {'know': float((kcm[1]+1)/2), 'can': float((kcm[2]+1)/2), 'master': float((kcm[0]+1)/2)}
    
    results['skills'] = skills


    # mod_descr = np.array([nlp_classic(description['descriptor']).vector])
    
    # predicted = list(kcm_extraction_model.predict(mod_descr)[0])
    # print(predicted[1])
    # results['skills'] = {description['descriptor']: {'know': float(
    #     predicted[1]), 'can': float(predicted[2]), 'master': float(predicted[0])}}

    return results

In [6]:
abc = 'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные характеристики.'

In [7]:
wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(abc, lambda text: nlp_classic(text).vector)


In [8]:
wp_of_event

array(['В форме', 'В форме представлений', 'В форме представлений типов',
       'В форме представлений типов мышления,',
       'В форме представлений типов мышления, обсуждение',
       'В форме представлений типов мышления, обсуждение их',
       'В форме представлений типов мышления, обсуждение их особенностей,',
       'В форме представлений типов мышления, обсуждение их особенностей, применительности',
       'В форме представлений типов мышления, обсуждение их особенностей, применительности к',
       'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным',
       'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам,',
       'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные',
       'форме представлений', 'форме представлений типов',
       'форме представлений типов мышления,',
       'форме представлений типов мышления, 

In [9]:
wp_of_event_vectorized

array([[ 0.14884901,  1.0217887 ,  1.1495686 , ...,  0.49929726,
        -1.3243097 , -0.8006755 ],
       [-0.3089898 ,  0.18650205,  0.44115317, ...,  0.20928188,
        -1.2772053 , -0.2967298 ],
       [-0.29398203, -0.20824577,  0.32792857, ...,  0.80966175,
        -1.2947845 , -0.5294124 ],
       ...,
       [ 0.00530585,  0.79299146, -1.3366222 , ..., -0.71472806,
         0.75286573,  0.85845184],
       [ 0.92110515,  0.45895028, -1.1853316 , ..., -0.33587068,
         0.53593826,  0.39650992],
       [ 1.3056835 ,  0.00824932, -0.7193653 , ..., -0.42750835,
         0.7859115 ,  0.26308134]], dtype=float32)

# Predict


In [10]:
is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized) > 0.992

3/3 [==============================] - 0s 2ms/step


In [11]:
is_wp_descriptor

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [

In [12]:
descriptors_vectorized = wp_of_event_vectorized[is_wp_descriptor.T[0]]
descriptors_vectorized[0]

array([-0.59135526, -0.18395996,  0.17253928, -0.04396268,  0.9353698 ,
        0.18774764,  1.2401947 ,  0.4095701 ,  0.21220809, -0.9230838 ,
       -0.15536904, -1.2114451 , -1.3029203 ,  1.4868159 , -0.28237996,
       -0.1745209 , -0.43754986, -0.73927516, -1.568915  ,  1.3296856 ,
        0.5218561 , -0.6542479 ,  0.6036532 ,  1.0517882 , -0.8474532 ,
       -0.20859516,  0.22622108, -0.43271026,  0.29257143, -0.5356993 ,
        0.34807053, -0.7714756 , -0.8102071 ,  0.6690045 , -0.3225636 ,
        0.3268718 , -2.135021  , -0.49551725,  0.2524806 , -0.8629706 ,
       -0.41905904,  0.43737972,  0.2783629 , -0.8816537 , -0.44250044,
       -1.4522643 , -0.2226783 , -1.0433569 , -0.5548043 ,  0.9303481 ,
       -0.2740887 ,  0.09103168, -0.06262046, -0.5992047 ,  1.4300972 ,
        0.28260157, -0.5352357 ,  0.55341023, -0.694973  ,  1.1642292 ,
       -0.20101662, -0.21450822,  0.11587569,  0.22602427,  1.0045058 ,
        0.16885489, -0.24806218,  0.30139938,  0.1086941 , -0.00

In [13]:
descriptors_vectorized[1]

array([-0.24248755,  0.14670034, -0.1305572 , -0.32413998,  1.2429131 ,
       -0.28657526,  1.2498873 ,  0.4367144 ,  0.18884082, -0.18016525,
        0.06810894, -0.7797362 , -0.9757614 ,  1.1603906 , -0.5705279 ,
        0.3956014 , -0.3727056 , -0.68488693, -1.335954  ,  1.1360757 ,
       -0.07043591, -0.7658009 ,  1.2543192 ,  0.7053085 , -0.9290271 ,
        0.41812205,  0.21950833, -1.1359097 ,  0.43440977, -0.35469288,
       -0.0896173 , -0.470287  , -0.7303042 ,  0.5891822 , -0.39858955,
       -0.07399984, -1.527461  ,  0.06778898,  0.07911275, -0.5422751 ,
       -0.6002258 ,  0.83571273, -0.10585116, -0.76372564,  0.01860676,
       -0.59096795,  0.19528525, -0.51249826, -0.40612116,  0.42418775,
       -0.1997738 ,  0.05652312, -0.38685614, -0.44971043,  1.8951824 ,
       -0.10483863,  0.26736927,  0.25721738, -0.77405584,  0.5931255 ,
       -0.21919087, -0.6393356 ,  0.08338185,  0.00899181,  0.85175544,
       -0.04897703,  0.16323695, -0.4722249 ,  0.5420947 , -0.03

In [14]:
predict = kcm_extraction_model.predict(descriptors_vectorized)
predict

1/1 [==============================] - 0s 55ms/step


array([[ 0.10309447,  0.9162789 ,  0.05999305],
       [ 0.44654724,  0.8315545 , -0.03878129],
       [-0.04274354,  1.1837949 ,  0.03735846],
       [-0.08032931,  1.152187  ,  0.17248598],
       [ 0.08650857,  0.79043174,  0.11069547],
       [ 0.395946  ,  0.7961303 , -0.05059106],
       [-0.13199744,  1.2420903 ,  0.0695568 ],
       [-0.14997078,  1.2672441 ,  0.23412174],
       [ 0.08551542,  1.0018843 ,  0.2685242 ],
       [-0.20891142,  1.3898848 ,  0.21903911]], dtype=float32)

In [15]:
descriptors_name = wp_of_event[is_wp_descriptor.T[0]]
descriptors_name

array(['форме представлений типов', 'форме представлений типов мышления,',
       'форме представлений типов мышления, обсуждение',
       'форме представлений типов мышления, обсуждение их',
       'представлений типов', 'представлений типов мышления,',
       'представлений типов мышления, обсуждение',
       'представлений типов мышления, обсуждение их', 'типов мышления,',
       'типов мышления, обсуждение'], dtype='<U116')

In [16]:
type(descriptors_name)

numpy.ndarray

## Distance

In [17]:
from string import punctuation

In [18]:
import re
import string

In [19]:
def desc_without_punct(descriptors_name):
    desc = list()
    for i in descriptors_name:
        desc.append(i.translate(str.maketrans('','', string.punctuation)))
    return desc

In [20]:
description_name_without_punct = desc_without_punct(descriptors_name)
description_name_without_punct

['форме представлений типов',
 'форме представлений типов мышления',
 'форме представлений типов мышления обсуждение',
 'форме представлений типов мышления обсуждение их',
 'представлений типов',
 'представлений типов мышления',
 'представлений типов мышления обсуждение',
 'представлений типов мышления обсуждение их',
 'типов мышления',
 'типов мышления обсуждение']

In [21]:
def vectorized_description_name_without_punct(description_name_without_punct):
    disc = list()
    for i in description_name_without_punct:
        disc.append(nlp_classic(i).vector)
    return disc

In [22]:
vectorized_description_name_without_punctuation = vectorized_description_name_without_punct(description_name_without_punct)

In [23]:
from scipy.spatial.distance import cosine

In [24]:
cosine(descriptors_vectorized[0], descriptors_vectorized[1])

0.12873321771621704

In [25]:
import itertools

In [26]:
n = 0
final_summ = [list() for i in vectorized_description_name_without_punctuation]

for i in range(len(vectorized_description_name_without_punctuation)):
    for j in range(i + 1, len(vectorized_description_name_without_punctuation)):
        cos = cosine(vectorized_description_name_without_punctuation[i], vectorized_description_name_without_punctuation[j])
        final_summ[i].append(cos)
        final_summ[j].append(cos)
    # final_summ.append(summator)
    # print("FN_inside", final_summ)

# print("FN", min(final_summ[0: -1]))
# res = np.where(final_summ[0: -1] == np.min(final_summ[0: -1]))[0]
# print(int(res))

print(final_summ )

[[0.07023108005523682, 0.1311246156692505, 0.2123168706893921, 0.17148226499557495, 0.2044416069984436, 0.2588135004043579, 0.3516587018966675, 0.23743152618408203, 0.3146370053291321], [0.07023108005523682, 0.05568045377731323, 0.1347089409828186, 0.20592910051345825, 0.08426719903945923, 0.14111000299453735, 0.23759084939956665, 0.11784225702285767, 0.1976313591003418], [0.1311246156692505, 0.05568045377731323, 0.0668189525604248, 0.2502344846725464, 0.1273554563522339, 0.059721291065216064, 0.14673733711242676, 0.16643977165222168, 0.08255219459533691], [0.2123168706893921, 0.1347089409828186, 0.0668189525604248, 0.313079833984375, 0.1905800700187683, 0.11203783750534058, 0.05058562755584717, 0.21841096878051758, 0.11687034368515015], [0.17148226499557495, 0.20592910051345825, 0.2502344846725464, 0.313079833984375, 0.10663294792175293, 0.18293797969818115, 0.2752116918563843, 0.20180588960647583, 0.29433900117874146], [0.2044416069984436, 0.08426719903945923, 0.1273554563522339, 0.1

In [27]:
def stdotkl(final_summ):
    final_list = list()
    for i in final_summ:
        final_list.append(np.std(i))
    return final_list

In [28]:
vectorize_end = stdotkl(final_summ)
vectorize_end

[0.08232651172542393,
 0.06034234802660114,
 0.05963460215067571,
 0.07881389511237764,
 0.06254702252910689,
 0.04817985473765189,
 0.06405746458673661,
 0.09205467920032423,
 0.05269335653147445,
 0.08671585664138665]

In [29]:
res = np.where(vectorize_end == np.min(vectorize_end))[0]

In [30]:
description_name_without_punct[int(res)]

'представлений типов мышления'

In [31]:
# skills = dict()
# for name, kcm in zip(descriptors_name, predict):
#         skills[name] = {'know': float((kcm[1]+1)/2), 'can': float((kcm[2]+1)/2), 'master': float((kcm[0]+1)/2)}
#         print(name, skills[name])

description_name_without_punct[int(res)], kcm_extraction_model.predict(np.array([nlp_classic(description_name_without_punct[int(res)]).vector]))



1/1 [==============================] - 0s 33ms/step


('представлений типов мышления',
 array([[ 0.3524445 ,  0.82012445, -0.16171165]], dtype=float32))